*I hereby thank ChatGPT for helping me learning and giving me ideas on how to play around the code*

*Note that my code will be formatted inconsistently--I'm still figuring out the best style I could use as I'm doing this project*

In [1]:
import pandas as pd
import numpy as py
import re

# b3a_dl1 *(Education)*

> ### *Importing & Sightseeing the Data*

In [3]:
b3a_dl1 = pd.read_stata("D:/College/IFLS 5/b3a_dl1.dta")

In [6]:
b3a_dl1 = b3a_dl1.rename(columns = {'hhid14': 'Household ID', 'pidlink': 'Person ID', 'dl06': 'Education'})


In [7]:
b3a_dl1.set_index('Person ID', inplace = True)

In [8]:
#b3a_dl1.head()

In [9]:
#b3a_dl1.loc['001060001':,'Education'].value_counts()

In [10]:
# Regular expression to match non-alphabetical characters
regex_pattern = r'[^a-zA-Z\s]'

#Remove the 'number:' at the beginning
b3a_dl1['Education'] = b3a_dl1['Education'].str.replace(regex_pattern, '', regex=True, n=3)

In [11]:
#b3a_dl1.loc[:, ['Household ID', 'Education']].head(25)

> ### *Transforming the Education Variable to Year of Education*

In [28]:
b3a_dl1.loc[:, 'Education'].value_counts()

Education
Elementary school                                   9896
Senior high general                                 5341
Junior high general                                 5124
Senior high vocational                              4546
University S1                                       3192
College (D1,D2,D3)                                  1226
Islamic Junior/High School (Madrasah Tsanawiyah)    1100
Islamic Senior/High School (Madrasah Tsanawiyah)     909
Islamic Elementary School (Madrasah Ibtidaiyah)      403
Adult education C                                    240
Junior high vocational                               206
University S2                                        202
Adult education B                                    108
Islamic School (pesantren)                            78
Open university                                       28
Adult education A                                     22
Other                                                 16
School for Disabled  

*educ = base year (dl06) + grades (dl07)*

*base year is the year of education an individual must have before enrolling to said education*

*grades are the highest grade ever achieved*

In [29]:
# BASE YEAR (base_educ) 
# making the base year ()
base_educ = {'Elementary school': 0,
             'Islamic Elementary School (Madrasah Ibtidaiyah)': 0,
             'Adult education A': 0,
             'Junior high general': 6,
             'Islamic Junior/High School (Madrasah Tsanawiyah)': 6,
             'Junior high vocational': 6,
             'Adult education B': 6,
             'Senior high general': 9,
             'Islamic Senior/High School (Madrasah Tsanawiyah)': 9,
             'Senior high vocational': 9,
             'Adult education C': 9,
             'University S1': 12,
             'College (D1,D2,D3)': 12,
             'Open university': 12,
             'University S2': 16,
             'University S3': 18 }

# mapping the base_educ to the Education series in the dataframe b3a_dl1
b3a_dl1['base_educ'] = b3a_dl1['Education'].map(base_educ)

In [30]:
b3a_dl1.loc[:,'dl07'].value_counts()
# turns out, 7 and 0 are string, and that would be problematic for my current method. thus, we need to modify dl07

dl07
7:Graduated                                     21129
2.0                                              3192
1.0                                              2849
0:Did not complete first grade at that level     2080
3.0                                              1265
4.0                                              1143
5.0                                               817
98:Don't Know                                     118
6.0                                                 4
99:MISSING                                          2
Name: count, dtype: int64

In [31]:
# GRADES (yeareduc, modification of dl07)

#We'll use "APPLY"

b3a_dl1['yeareduc'] = b3a_dl1['dl07']

#first, we define what would be applied to 'yeareduc'
def anomaly(row):
    if row['dl07'] in [0, 1, 2, 3, 4, 5, 6]:
        return row['yeareduc']
    elif row['dl07'] == '7:Graduated':
        if row['Education'] in ['Elementary school', 'Islamic Elementary School (Madrasah Ibtidaiyah)', 'Adult education A']:
            return 6
        elif row['Education'] in ['Junior high general', 'Islamic Junior/High School (Madrasah Tsanawiyah)', 'Junior high vocational', 'Adult education B', 'Senior high general', 'Islamic Senior/High School (Madrasah Tsanawiyah)', 'Senior high vocational', 'Adult education C']:
            return 3
        elif row['Education'] in ['University S1', 'College (D1,D2,D3)', 'Open university']:
            return 4
        elif row['Education'] in ['University S2', 'University S3']:
            return 2
    elif row['dl07'] == '0:Did not complete first grade at that level':
        return 0
    return None

b3a_dl1['yeareduc'] = b3a_dl1.apply(anomaly, axis=1)


In [32]:
# BASE + GRADES
b3a_dl1['educ'] = b3a_dl1['yeareduc'] + b3a_dl1['base_educ']

education = b3a_dl1['educ'] # to be merged (pid)

# its not checked with stata since the method is a bit different, but i'm confident on this one more

> ### *Checking*

In [34]:
b3a_dl1.loc[:,['educ', 'base_educ', 'yeareduc', 'dl07', 'Education']].head(50)

,educ,base_educ,yeareduc,dl07,Education
Person ID,,,,,
001060001,4.0,0.0,4.0,4.0,Elementary school
001060007,0.0,0.0,0.0,0:Did not complete first grade at that level,Elementary school
001060008,7.0,6.0,1.0,1.0,Junior high general
001060004,3.0,0.0,3.0,3.0,Elementary school
001065102,8.0,6.0,2.0,2.0,Junior high general
001080003,1.0,0.0,1.0,1.0,Elementary school
001080012,NaN,NaN,NaN,NaN,NaN
001080008,1.0,0.0,1.0,1.0,Elementary school
001220001,4.0,0.0,4.0,4.0,Elementary school


In [21]:
b3a_dl1.loc[:,'educ'].value_counts()

educ
12.0    8700
6.0     5536
9.0     5333
16.0    3142
2.0     1042
4.0     1014
3.0      989
8.0      951
10.0     942
11.0     855
5.0      779
1.0      772
7.0      768
0.0      459
13.0     337
14.0     323
15.0     270
18.0     164
17.0      65
20.0       5
19.0       2
22.0       1
Name: count, dtype: int64

> > >*SOME LEFTOVER COMMANDS FOR FUTURE REFERENCE*

In [17]:
#Now, I want to create a series that is add-able to make base year + grade (dl07)
#grades_a = [0, 1, 2, 3, 4, 5, 6] #since dtype is int, we dont use ''
#grades = b3a_dl1['dl07'].isin(grades_a)
#grades.value_counts()

dl07
False    25194
True      9270
Name: count, dtype: int64

In [18]:
#now, for the graduated, similarly:
#grades_b = ['7:Graduated'] #NOTICE THAT THIS IS STRING
#graduates = b3a_dl1['dl07'].isin(grades_b)
#graduates.value_counts()

dl07
True     21129
False    13335
Name: count, dtype: int64

In [19]:
#generating new variable 'yeareduc' where if dl07's values is chosen (in grades_a), the value will equal dl07, otherwise None:

#this one is for ones that are add-able
#if b3a_dl1['dl07'].isin(grades_a).any():
#    b3a_dl1['yeareduc'] = py.where(grades, b3a_dl1['dl07'], None)

#for the graduate
#sdx = ['Elementary school', 'Islamic Elementary School (Madrasah Ibtidaiyah)', 'Adult education A']
#sd = b3a_dl1['Education'].isin(sdx)

#smpsmax = ['Junior high general', 'Islamic Junior/High School (Madrasah Tsanawiyah)', 'Junior high vocational', 'Adult education B', 'Senior high general', 'Islamic Senior/High School (Madrasah Tsanawiyah)', 'Senior high vocational', 'Adult education C']
#smpsma = b3a_dl1['Education'].isin(smpsmax)

#bachelorx = ['University S1', 'College (D1,D2,D3)', 'Open university']
#bachelor = b3a_dl1['Education'].isin(bachelorx)

#profx = ['University S2', 'University S3']
#prof = b3a_dl1['Education'].isin(profx)


In [209]:
#this was a mistake and the command was flawed

#if b3a_dl1['dl07'].isin(grades_b).any() and b3a_dl1['Education'].isin(sdx).any():
   # b3a_dl1['yeareduc'] = py.where(graduates & sd, 6, None)
#elif b3a_dl1['dl07'].isin(grades_b).any() and b3a_dl1['Education'].isin(smpsma).any():
#    b3a_dl1['yeareduc'] = py.where(graduates & smpsma, 3, None)
#elif b3a_dl1['dl07'].isin(grades_b).any() and b3a_dl1['Education'].isin(bachelor).any():
#    b3a_dl1['yeareduc'] = py.where(graduates & bachelor, 4, None)
#elif b3a_dl1['dl07'].isin(grades_b).any() and b3a_dl1['Education'].isin(prof).any():
 #   b3a_dl1['yeareduc'] = py.where(graduates & prof, 2, None)

# b1_ks1 *(Consumptions)*

> ### *Importing & Sightseeing the Data*

In [35]:
b1_ks1 = pd.read_stata("D:/College/IFLS 5/b1_ks1.dta")

In [36]:
#b1_ks1.head(5)

In [37]:
#b1_ks1.info()

In [38]:
b1_ks1.set_index('hhid14', inplace=True)

In [39]:
###create a series to check duplicate of ks1type in each household
    #b1_ks1['ks1type_duplicate'] = b1_ks1.groupby(b1_ks1.index)['ks1type'].transform(lambda x: x.duplicated())
#randomly check a household
    #b1_ks1.loc['0010600',['ks1type', 'ks02', 'food', 'ks1type_duplicate']]
#seems okay

> ### *Making the protein etc. variables*

In [40]:
#equivalent to "keep ks02 hhid14 ks1type"
foodtype = b1_ks1.loc[:,['ks1type', 'ks02']]

In [41]:
#equivalent to "reshape wide ks02, i (hhid14) j (ks1type) string"
food = foodtype.pivot_table(index='hhid14', columns='ks1type', values='ks02', aggfunc='sum', fill_value=0)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Total Food

In [42]:
food['totalfood'] = food[food.columns].sum(axis=1)
#be sure to execute this whilst the df food is untouched after pivoting (previous line)

In [43]:
#food.loc['3212541','totalfood']
#VERIFIED WITH STATA

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Protein & Fiber

In [44]:
food['protein'] = food[['K', 'L', 'M', 'N', 'OA', 'OB', 'P', 'Q']].sum(axis=1) #axis=1 means its row-summing, =0 or default is column summing
#VERIFIED WITH STATA! 

In [45]:
food['fiber'] = food[['F', 'G', 'H']].sum(axis=1)
#VERIFIED WITH STATA! 

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Healthy Eating Index (HEI)

In [2]:
#Food Proportion
food['fishmeat']=food['K']+food['L']+food['M']+food['N']+food['OA']
food['sugar']=food['AA']+food['W']
food['saturatedfat']=food['K']+food['Y']+food['X']+food['Q']

for column in food.columns:
    if column not in ['protein', 'fiber', 'totalfood']:
        food[f'{column}_proportion'] = food[column]/food['totalfood']



adequacyfood = food.quantile(0.25)
moderationfood = food.quantile(0.75)

NameError: name 'food' is not defined

> *Adequacy & Moderation*

In [47]:
for column in food.columns:
    #adequacy
   if column in ['F_proportion', 'H_proportion', 'Q_proportion', 'fishmeat_proportion', 'G_proportion', 'Y_proportion']:
      food[f'{column}_adequacy'] = py.where(food[column] > adequacyfood.loc[column], 1, 0)
    #moderation
   if column in ['EA_proportion', 'FA_proportion', 'sugar_proportion', 'S_proportion', 'saturatedfat_proportion']:
      food[f'{column}_moderation'] = py.where(food[column] <= moderationfood.loc[column], 1, 0)

> *HEI*

In [48]:
HEI_columns = [col for col in food.columns if col.endswith('_adequacy') or col.endswith('_moderation')]

# Create new series as sum of the identified columns
food['HEI'] = food[HEI_columns].sum(axis=1)

#it yields different result than stata since (in 'Moderation'), the quantile of EA and FA is 0, and thus when casted < moderation.loc[column] (or < 0), it will always show 0

In [49]:
#if fail:

###todrop = [col for col in food.columns if col.endswith('_proportion') or col.endswith('_adequacy') or col.endswith('_moderation')]
###food = food.drop(columns=todrop)

#del food

> *CHECKING*

In [106]:
for col in food.columns:
    if col=='HEI' or col.endswith('_adequacy') or col.endswith('_moderation'):
        print(food.loc[:,[f'{col}']].value_counts()) #DONT FORGET THE PRINT IN LOOPS OTHERWISE IT WONT SHOW SHIT

EA_proportion_moderation
1                           11509
0                            3635
Name: count, dtype: int64
F_proportion_adequacy
1                        11301
0                         3843
Name: count, dtype: int64
FA_proportion_moderation
1                           14909
0                             235
Name: count, dtype: int64
G_proportion_adequacy
0                        11688
1                         3456
Name: count, dtype: int64
H_proportion_adequacy
1                        9026
0                        6118
Name: count, dtype: int64
Q_proportion_adequacy
0                        8519
1                        6625
Name: count, dtype: int64
S_proportion_moderation
1                          11302
0                           3842
Name: count, dtype: int64
Y_proportion_adequacy
1                        10821
0                         4323
Name: count, dtype: int64
fishmeat_proportion_adequacy
1                               11301
0                                

In [51]:
nutrition = food[['HEI', 'protein', 'fiber']] #to be merged (CAREFUL! identifier is hhid14)

In [52]:
nutrition

ks1type,HEI,protein,fiber
hhid14,,,
0010600,7,172000.0,5000.0
0010651,5,45000.0,0.0
0010800,6,18000.0,0.0
0010851,7,15000.0,10000.0
0012200,9,296000.0,30000.0
...,...,...,...
3212852,9,80500.0,46500.0
3212853,9,40000.0,70000.0
3212900,9,179000.0,24000.0


> *SOME LEFTOVER COMMANDS FOR FUTURE REFERENCE*

In [214]:
#THIS IS TO DIVIDE SERIES INTO SEVERAL PARTS: Q=4 QUANTILES, Q=10 DECILES, AND SO ON

#for column in food.columns:
 #   if column in ['F_proportion', 'H_proportion', 'Q_proportion', 'fishmeat_proportion', 'G_proportion', 'Y_proportion']:
   #     #food[f'{column}_adequacy'] = py.where(food[column] > adequacy.loc[column], 1, 0)
    #    food[f'{column}_adequacy'] = pd.qcut(food[f'{column}'], q=100, labels=False)
     #   food['HEI'] = food[f'{column}_adequacy'].apply(lambda x: 1 if 75 > x >= 100 else 0)


In [181]:
#THIS IS TRY-EXCEPT NESTED

#for column in food.columns:
 #   if column in ['F_proportion', 'H_proportion', 'Q_proportion', 'fishmeat_proportion', 'G_proportion', 'Y_proportion']:
  #     # food[f'{column}_adequacy'] = py.where(food[column] > adequacy.loc[column], 1, 0)
   #     try:
    #        try:
     #           food[f'{column}_adequacy'] = pd.qcut(food[f'{column}'], q=4, labels=False)
      #      except ValueError as e:
       #         food[f'{column}_adequacy'] = pd.qcut(food[f'{column}'], q=3, labels=False)
        #except ValueError as e:
         #   try:
          #      food[f'{column}_adequacy'] = pd.qcut(food[f'{column}'], q=2, labels=False)
           # except ValueError as e:
            #    food[f'{column}_adequacy'] = pd.qcut(food[f'{column}'], q=1, labels=False)
    #    food['HEI'] = food[f'{column}_adequacy'].apply(lambda x: 1 if x in [0, 1, 2] else 0)


In [213]:
#THIS IS FOR EACH IN STATA

#for column in food.columns:
 #   if column in ['F_proportion', 'H_proportion', 'Q_proportion', 'fishmeat_proportion', 'G_proportion', 'Y_proportion']:
  #      print(food.loc[:,f'{column}_adequacy'].value_counts().to_frame())

# bk_ar1 *(Income, Gender, Age, Marital Status, Fam Educ)*

In [2]:
bk_ar1 = pd.read_stata("D:/College/IFLS 5/bk_ar1.dta")
bk_ar1.set_index("pidlink", inplace=True)

In [3]:
bk_ar1_a = bk_ar1[['ar01a', "ar15b", "ar15bx", "ar02b", "ar07", "ar09", "ar13", "ar16", "ar17", "ar15a", "hhid14"]].copy()

bk_ar1_a

,ar01a,ar15b,ar15bx,ar02b,ar07,ar09,ar13,ar16,ar17,ar15a,hhid14
pidlink,,,,,,,,,,,
001060001,1.0,NaN,NaN,1:Head of the household,1.0,59.0,2:Married,2:Grade school,4,3:No,0010600
001060002,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010600
001060003,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010600
001060004,3.0,NaN,8:Don't know,3:Children (biological,3.0,29.0,2:Married,2:Grade school,0:Did not finish 1st grade,1:Yes,0010600
001060005,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010600
...,...,...,...,...,...,...,...,...,...,...,...
321291105,1.0,NaN,NaN,3:Children (biological,3.0,12.0,1:Unmarried,73:Madrasah Tsanawiyah,0:Did not finish 1st grade,3:No,3212911
321300001,1.0,7000000.0,1:What was HHM's income from work in the last ...,1:Head of the household,1.0,48.0,2:Married,6:Vocational sr. high (SMK),7:Graduated,1:Yes,3213000
321300002,1.0,NaN,NaN,2:Husband/wife,3.0,58.0,2:Married,5:General sr. high (SLA),7:Graduated,3:No,3213000


> ### *Household Size* 

In [4]:
# making household size variable
bk_ar1_h = bk_ar1_a.copy()
x = pd.Series(1, index=bk_ar1_h.index)
bk_ar1_h['x']=x

In [5]:
hhsizecon = bk_ar1_h.loc[
            (bk_ar1_h['ar01a']==1) | 
            (bk_ar1_h['ar01a']==5) | 
            (bk_ar1_h['ar01a']==11)
            ]

In [6]:
hhsizecon

,ar01a,ar15b,ar15bx,ar02b,ar07,ar09,ar13,ar16,ar17,ar15a,hhid14,x
pidlink,,,,,,,,,,,,
001060001,1.0,NaN,NaN,1:Head of the household,1.0,59.0,2:Married,2:Grade school,4,3:No,0010600,1
001060007,5.0,NaN,6:Unpaid family worker,2:Husband/wife,3.0,39.0,2:Married,1:Unschooled,96:No school,1:Yes,0010600,1
001060008,5.0,NaN,NaN,3:Children (biological,3.0,16.0,1:Unmarried,3:General jr. high,1,3:No,0010600,1
001060009,5.0,NaN,NaN,3:Children (biological,3.0,12.0,1:Unmarried,2:Grade school,4,3:No,0010600,1
001060010,5.0,NaN,NaN,3:Children (biological,1.0,3.0,1:Unmarried,1:Unschooled,96:No school,6:< 5 Year,0010600,1
...,...,...,...,...,...,...,...,...,...,...,...,...
321291104,1.0,NaN,NaN,3:Children (biological,1.0,19.0,1:Unmarried,61:University S1,0:Did not finish 1st grade,3:No,3212911,1
321291105,1.0,NaN,NaN,3:Children (biological,3.0,12.0,1:Unmarried,73:Madrasah Tsanawiyah,0:Did not finish 1st grade,3:No,3212911,1
321300001,1.0,7000000.0,1:What was HHM's income from work in the last ...,1:Head of the household,1.0,48.0,2:Married,6:Vocational sr. high (SMK),7:Graduated,1:Yes,3213000,1


In [7]:
# making household size variable
hhsize = hhsizecon.groupby('hhid14')['x'].sum().reset_index()

In [8]:
hhsize.rename(columns={'x':'hhsize'}, inplace=True)

In [9]:
hhsize.set_index('hhid14', inplace=True) 

In [10]:
hhsize.describe()
#to be merged (CAREFUL: HHID14)
#VERIFIED WITH STATA CHECKING

,hhsize
count,14650.000000
mean,3.469761
std,1.742662
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,15.000000


> ### *Gender*

In [11]:
bk_ar1_g = bk_ar1_a.copy()
bk_ar1_g

,ar01a,ar15b,ar15bx,ar02b,ar07,ar09,ar13,ar16,ar17,ar15a,hhid14
pidlink,,,,,,,,,,,
001060001,1.0,NaN,NaN,1:Head of the household,1.0,59.0,2:Married,2:Grade school,4,3:No,0010600
001060002,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010600
001060003,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010600
001060004,3.0,NaN,8:Don't know,3:Children (biological,3.0,29.0,2:Married,2:Grade school,0:Did not finish 1st grade,1:Yes,0010600
001060005,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010600
...,...,...,...,...,...,...,...,...,...,...,...
321291105,1.0,NaN,NaN,3:Children (biological,3.0,12.0,1:Unmarried,73:Madrasah Tsanawiyah,0:Did not finish 1st grade,3:No,3212911
321300001,1.0,7000000.0,1:What was HHM's income from work in the last ...,1:Head of the household,1.0,48.0,2:Married,6:Vocational sr. high (SMK),7:Graduated,1:Yes,3213000
321300002,1.0,NaN,NaN,2:Husband/wife,3.0,58.0,2:Married,5:General sr. high (SLA),7:Graduated,3:No,3213000


In [12]:
#change into int
bk_ar1_g['ar07'] = bk_ar1_a['ar07'].astype('Int64')

#replace 3 == 0
bk_ar1_g['ar07'] = bk_ar1_a['ar07'].replace(3, 0)

gender = bk_ar1_g['ar07']
gender = gender.to_frame()
gender #to be merged

,ar07
pidlink,
001060001,1.0
001060002,NaN
001060003,NaN
001060004,0.0
001060005,NaN
...,...
321291105,0.0
321300001,1.0
321300002,0.0


> ### *Income*

In [13]:
incomedf = bk_ar1_a[['ar15a', 'ar15b', 'ar15bx', 'ar02b', 'ar07', 'hhid14']].copy()

In [14]:
income = incomedf.copy()

> > ### *Per Person Income*

In [15]:
######### NORMAL HUMAN METHOD

# convert to int (use income= at first to use this command) - ignore errors:
#income['ar15a'] = income['ar15a'].str.extract(r'(\d+)').astype('Int64')
#income['ar15bx'] = income['ar15bx'].str.extract(r'(\d+)').astype('Int64')
#income['ar02b'] = income['ar02b'].str.extract(r'(\d+)').astype('Int64')

# r'(\d)') is sort of 'catcher' for digits 0-9, and would terminate once it founds the first digit
# adding + after d would make it such that it would catch consecutive digits

#--#

######### ANOTHER METHOD:

# using apply (THIS IS WRONG SINCE IT CATCHES ALL DIGITS:
#def extract_digits(s):
  #  if isinstance(s, str):
   #     digits = ''.join(filter(str.isdigit, s)) #this returns the first consecutive (if any) digits
    #    return int(digits) if digits else py.nan #return digits in int form, otherwise NA
   # return py.nan

In [16]:
#THIS IS CORRECT (ALTHOUGH A BIT INHUMAN)
def extract_digit(s):
    if isinstance(s, str):
        match = re.search(r'\b\d+\b', s) # finds for the first consecutive digits
        if match: # if match found
            return int(match.group()) # return the integer version of match.group(), which is matched digits as string
    return py.nan
    
income[['ar15a', 'ar15bx', 'ar02b']] = incomedf[['ar15a', 'ar15bx', 'ar02b']].map(extract_digit)

#i can actually just skip this... if i put the next command with string instead of int..
#i mean good for me i learnt something new but fuck me was that unnecessary

In [17]:
income.loc[(income['ar15bx']==6) & (income['ar15b']!=py.nan), 'ar15b'] = 0 # updated with & 'ar15bx'!=py.nan, haven't been run/checked yet
income.loc[(income['ar15a']==3) & (income['ar15b']!=py.nan), 'ar15b'] = 0 # updated with & 'ar15bx'!=py.nan, haven't been run/checked yet
income.loc[income['ar15b']>999999990, 'ar15b'] = py.nan

In [18]:
income = income['ar15b']

In [19]:
income = income.to_frame()
income # to be merged, verified with stata

,ar15b
pidlink,
001060001,0.0
001060002,NaN
001060003,NaN
001060004,NaN
001060005,NaN
...,...
321291105,0.0
321300001,7000000.0
321300002,0.0


> > ### *Head of the Family Income*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Stating which income is whose:

In [20]:
incomehead = incomedf.copy()

In [21]:
incomehead['head'] = income

In [22]:
incomehead.loc[incomehead['ar02b']!='1:Head of the household', 'head'] = py.nan

In [23]:
incomehead = incomehead['head']

In [24]:
incomehead = incomehead.to_frame()
incomehead #to be merged #verified stata

,head
pidlink,
001060001,0.0
001060002,NaN
001060003,NaN
001060004,NaN
001060005,NaN
...,...
321291105,NaN
321300001,7000000.0
321300002,NaN


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Stating HF income per Household ID (hhid14)

In [25]:
income_head = incomedf.copy()

In [26]:
income_head['incomeheadPID'] = incomehead

In [27]:
income_head['hhid14'] = bk_ar1['hhid14']

In [28]:
income_head = income_head.groupby('hhid14')['incomeheadPID'].max()

In [29]:
income_head = income_head.to_frame()
income_head #to be merged 
#CAREFUL: ON HHID14!

,incomeheadPID
hhid14,
0010600,0.0
0010651,2000000.0
0010800,3000000.0
0010851,6000000.0
0012200,36000000.0
...,...
3212852,270000000.0
3212853,55200000.0
3212900,354000000.0


In [30]:
#converting to logarithm 
income_head_nozero = income_head.replace(0, py.nan)
income_head_log = py.log(income_head_nozero)

In [31]:
income_head_log #to be merged 
#CAREFUL: ON HHID14!

,incomeheadPID
hhid14,
0010600,NaN
0010651,14.508658
0010800,14.914123
0010851,15.607270
0012200,17.399029
...,...
3212852,19.413933
3212853,17.826474
3212900,19.684807


> > ### *Per Capita Income (Average)*
*Careful that this is averaged, use 'income' instead for true value if possible*

In [32]:
incomepc = incomedf.copy()

In [33]:
# we merge the 'hhsize' information
income_pc = pd.merge(incomepc, hhsize, left_on='hhid14', right_index=True, how='left') #right_index=True here is setting the right_on to the right's index

In [34]:
# we sum the total income on a household
income_total = income_pc.groupby('hhid14')['ar15b'].sum().to_frame()
income_total

,ar15b
hhid14,
0010600,0.0
0010651,2000000.0
0010800,51000000.0
0010851,6000000.0
0012200,49620000.0
...,...
3212852,390000000.0
3212853,115200000.0
3212900,354000000.0


In [35]:
# we merge the summed data to the previous data so its pivoted back to the original (i.e. each person holds the information for the total income/household)
income_pc = pd.merge(income_pc, income_total, left_on='hhid14', right_index=True, how='left')

In [36]:
# renaming so i don't get confused
income_pc.rename(columns={'ar15b_y': 'income_total', 'ar15b_x': 'ar15b'}, inplace=True)

In [37]:
# creating the var
income_pc['income_pc'] = income_pc['income_total'] / income_pc['hhsize']
income_pc #to be merged (on pidlink)

,ar15a,ar15b,ar15bx,ar02b,ar07,hhid14,hhsize,income_total,income_pc
pidlink,,,,,,,,,
001060001,3:No,NaN,NaN,1:Head of the household,1.0,0010600,6.0,0.0,0.000000e+00
001060002,NaN,NaN,NaN,NaN,NaN,0010600,6.0,0.0,0.000000e+00
001060003,NaN,NaN,NaN,NaN,NaN,0010600,6.0,0.0,0.000000e+00
001060004,1:Yes,NaN,8:Don't know,3:Children (biological,3.0,0010600,6.0,0.0,0.000000e+00
001060005,NaN,NaN,NaN,NaN,NaN,0010600,6.0,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...
321291105,3:No,NaN,NaN,3:Children (biological,3.0,3212911,5.0,19500000.0,3.900000e+06
321300001,1:Yes,7000000.0,1:What was HHM's income from work in the last ...,1:Head of the household,1.0,3213000,3.0,10000000.0,3.333333e+06
321300002,3:No,NaN,NaN,2:Husband/wife,3.0,3213000,3.0,10000000.0,3.333333e+06


> > ### *Parents' Income*

In [38]:
incomeparents = incomedf.copy()

In [39]:
incomeparents['incomeparents'] = income

In [40]:
incomeparents.loc[incomeparents['ar02b']!='2:Husband/wife', 'incomeparents'] = py.nan

In [41]:
incomeparents[['ar02b', 'incomeparents', 'ar15b']].head(25)

,ar02b,incomeparents,ar15b
pidlink,,,
001060001,1:Head of the household,NaN,NaN
001060002,NaN,NaN,NaN
001060003,NaN,NaN,NaN
001060004,3:Children (biological,NaN,NaN
001060005,NaN,NaN,NaN
001060006,NaN,NaN,NaN
001060007,2:Husband/wife,0.0,NaN
001060008,3:Children (biological,NaN,NaN
001060009,3:Children (biological,NaN,NaN


# CHECK IF AR15B = NaN THEN SO SHOULD BE FOLLOWED